# **Criando uma base de dados**

In [1]:
import pandas as pd
import numpy as np

treino= pd.read_csv("/content/train.csv")

treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#**Arrumando os dados**

**Excluindo Colunas**

In [2]:
exclui_coluna= ["Name", "Ticket", "Cabin", "Embarked", "Fare", "PassengerId"]
treino.drop(columns= exclui_coluna, inplace= True)

treino.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


**Transformando categorico em discreto**

In [3]:
treino["Sex"]= treino["Sex"].replace(["male", "female"], [0, 1])
treino["Sex"].head()

0    0
1    1
2    1
3    1
4    0
Name: Sex, dtype: int64

**Trantando Numeros Nulos**

In [4]:
treino.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
dtype: int64

In [5]:
# Age
def idade_mediana(df):
  mediana= df["Age"].median()
  df.update(df["Age"].fillna(mediana))
  return(df)

treino= idade_mediana(treino)
treino.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,0,22.0,1,0
1,1,1,1,38.0,1,0
2,1,3,1,26.0,0,0
3,1,1,1,35.0,1,0
4,0,3,0,35.0,0,0


**Separando o target da base de dados**

In [6]:
x= treino.iloc[:, 1:]
y= treino.iloc[:, 0:1]

#**Utilizando Auto-ML tpot**

In [ ]:
!pip install tpot

In [8]:
from tpot import TPOTClassifier

modelo_TPOT= TPOTClassifier(generations=8, population_size=100,
                            cv=5, n_jobs= -1, random_state=2, verbosity=2).fit(x, y)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Generation 1 - Current best internal CV score: 0.8226727763480006

Generation 2 - Current best internal CV score: 0.8226727763480006

Generation 3 - Current best internal CV score: 0.837266963781307

Generation 4 - Current best internal CV score: 0.837266963781307

Generation 5 - Current best internal CV score: 0.837266963781307

Generation 6 - Current best internal CV score: 0.837266963781307

Generation 7 - Current best internal CV score: 0.837266963781307

Generation 8 - Current best internal CV score: 0.837266963781307

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.1, max_depth=10, max_features=0.8, min_samples_leaf=18, min_samples_split=20, n_estimators=100, subsample=0.7000000000000001)


In [9]:
my_dict = list(modelo_TPOT.evaluated_individuals_.items())

model_scores = pd.DataFrame()
for model in my_dict:
    model_name = model[0]
    model_info = model[1]
    cv_score = model[1].get('internal_cv_score') 
    model_scores = model_scores.append({'model': model_name,
                                        'cv_score': cv_score,
                                        'model_info': model_info,},
                                       ignore_index=True)

model_scores = model_scores.sort_values('cv_score', ascending=False)
model_scores.head()

,cv_score,model,model_info
305,0.837267,"GradientBoostingClassifier(input_matrix, Gradi...","{'generation': 3, 'mutation_count': 2, 'crosso..."
821,0.835020,"XGBClassifier(input_matrix, XGBClassifier__lea...","{'generation': 8, 'mutation_count': 3, 'crosso..."
782,0.831687,RandomForestClassifier(CombineDFs(input_matrix...,"{'generation': 8, 'mutation_count': 4, 'crosso..."
535,0.831674,GradientBoostingClassifier(LinearSVC(input_mat...,"{'generation': 5, 'mutation_count': 1, 'crosso..."
824,0.831662,XGBClassifier(KNeighborsClassifier(input_matri...,"{'generation': 8, 'mutation_count': 2, 'crosso..."
